In [27]:
from math import sqrt
import numpy as np
from numpy import concatenate
from numpy import transpose
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import autocorrelation_plot
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LSTM
from keras.layers import Dropout, Softmax
from keras.optimizers import RMSprop, SGD
seed_value= 1

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)
# Get same result every time
#np.random.seed(10)
#tf.random.set_seed(10)
dataset = pd.read_csv('energydata_complete.csv', header = 0, sep=',', quotechar='"', engine='python', usecols=[0, 1], 
                           dtype = {"date" : "datetime64[ns]", "Appliances" : "float"})

#datetime format
dataset.index = pd.to_datetime(dataset['date'], format='%Y-%m-%d %H:%M:%S')
dataset = dataset.set_index('date')
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train1, test1 = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

def onestep_split(dataset,k=1):
    X, Y = [], []
    for i in range(len(dataset)-k-1): #(n -(k+1)) k=1
        a = dataset[i:(i+k), 0]
        X.append(a)
        Y.append(dataset[i + k, 0])
    return np.array(X), np.array(Y)
   

X_train, Y_train = onestep_split(train1, 1)
X_test, Y_test = onestep_split(test1, 1)

# the net needs to get the current time step and the actual value --> shape[1], shape[2]
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))



In [28]:
def create_model(optimizer, init='uniform'):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(1, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1))    
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [29]:
from keras.wrappers.scikit_learn import KerasRegressor
kears_estimator = KerasRegressor(build_fn=create_model, verbose=1)

/var/folders/tz/9k5bcs_502z2svd2fr_k2w680000gn/T/ipykernel_842/1061118482.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  kears_estimator = KerasRegressor(build_fn=create_model, verbose=1)


In [30]:
from sklearn.pipeline import Pipeline
estimator = Pipeline([ 
                       ("kc", kears_estimator)])

In [31]:
# define the grid search parameters
param_grid = {
    'kc__epochs': [10, 20],
    'kc__init': [ 'uniform', 'zeros', 'normal'], 
    'kc__batch_size':[32, 64],
    'kc__optimizer':['adam','sgd']
}

In [32]:
from sklearn.model_selection import GridSearchCV
kfold_splits = 5
grid = GridSearchCV(estimator=estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=kfold_splits,  #StratifiedKFold(n_splits=kfold_splits, shuffle=True)
                    param_grid=param_grid,
                    refit=True)

In [33]:
grid_result = grid.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-02 11:14:47.337716: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 11:14:47.340498: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-05-02 11:14:47.348740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 11:14:47.348869: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL,

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-02 11:14:47.581035: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 11:14:47.581145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-05-02 11:14:47.638708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 11:14:47.639484: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL,

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10


...

198/198 [==============================] - 1s 3ms/step - loss: 0.0042
Epoch 1/20


2023-05-02 11:39:53.230664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-02 11:39:53.390434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  9/247 [>.............................] - ETA: 1s - loss: 0.0115  

2023-05-02 11:39:53.584461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


247/247 [==============================] - 3s 7ms/step - loss: 0.0068
Epoch 2/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0043
Epoch 3/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0041
Epoch 4/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 5/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 6/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0041
Epoch 7/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 8/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0041
Epoch 9/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 10/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 11/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0041
Epoch 12/20
247/247 [==============================] - 2s 6ms/step - loss: 0.0042
Epoch 13/20
247/247 [===============

In [34]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.004087 using {'kc__batch_size': 64, 'kc__epochs': 20, 'kc__init': 'zeros', 'kc__optimizer': 'sgd'}
-0.004351 (0.000567) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'uniform', 'kc__optimizer': 'adam'}
-0.004213 (0.000453) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'uniform', 'kc__optimizer': 'sgd'}
-0.004206 (0.000405) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'zeros', 'kc__optimizer': 'adam'}
-0.004298 (0.000503) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'zeros', 'kc__optimizer': 'sgd'}
-0.004120 (0.000453) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'normal', 'kc__optimizer': 'adam'}
-0.004357 (0.000941) with: {'kc__batch_size': 32, 'kc__epochs': 10, 'kc__init': 'normal', 'kc__optimizer': 'sgd'}
-0.004195 (0.000474) with: {'kc__batch_size': 32, 'kc__epochs': 20, 'kc__init': 'uniform', 'kc__optimizer': 'adam'}
-0.004225 (0.000445) with: {'kc__batch_size': 32, 'kc__epochs': 20, 'kc__init': 'uniform'

In [35]:
from joblib import dump, load

estimator = grid.best_estimator_

dump(estimator, "GRIDLSTM.joblib")
# Somewhere else
#estimator = load("your-model.joblib")

INFO:tensorflow:Assets written to: ram://5a15d0d0-a405-488b-9891-28aa847d362b/assets


INFO:tensorflow:Assets written to: ram://5a15d0d0-a405-488b-9891-28aa847d362b/assets


['GRIDLSTM.joblib']